# Code 2

In [36]:
import plotly.graph_objects as go

def Plot_consom(times, PFC_values, tmin, tmax, PFCmin, PFCmax):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Add horizontal lines for PFCmin and PFCmax
    fig.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    
    fig.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")


    # Customize layout
    fig.update_layout(title='Consommation de fonte au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False,annotations=[
            dict(x=tmax,y=PFCmin,xref="x",yref="y",text="PFCmin",showarrow=True,arrowhead=2,ax=20,ay=-30),
            dict(x=tmax,y=PFCmax,xref="x",yref="y",text="PFCmax",showarrow=True,arrowhead=2,ax=20,ay=-30)])

    # Show the plot
    fig.show()

def Plot_Prod(times, PFC_values):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Customize layout
    fig.update_layout(title='Production de fonte au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False)

    # Show the plot
    fig.show()

In [48]:
def ProdConsom(ListeMasse_grappe, ListeQuantite_mottes, nb_moules_heure, t_serie, Temps_fusion, Temps_chargement):

    numfours = 0
    Fonte_produit = 5000 # Fontes produits toute les 45 min en kg
    PFC_Prodvalues = [0] # en kg
    PFC_Prod =  PFC_Prodvalues[-1]


    PFC_Consomvalues = [4000] # en kg
    PFC_Consom = PFC_Consomvalues[-1]
    times = [0] # en min
    tmin = times[-1]
    t = tmin

    for masse_grappe_i, Quantite_mottes_i in zip(ListeMasse_grappe, ListeQuantite_mottes) :
        for i in range(Quantite_mottes_i):
            cadences_i = nb_moules_heure/60 * masse_grappe_i  # consomation de fonte par minute (kg/min)
            PFC_Consom = PFC_Consom - cadences_i  # en kg
            t+=1
            times.append(t)
            PFC_Consomvalues.append(PFC_Consom)

            #  Partie Producion de la fonte dans le temps
            if t <= Temps_chargement:
                # Avant ou pendant Temps_chargement
                PFC_Prodvalues.append(PFC_Prod)
            else:
                # Après Temps_chargement
                if (t - Temps_chargement) % Temps_fusion == 0:
                    numfours += 1
                    # print(t,numfours)
                    PFC_Prod += Fonte_produit
                PFC_Prodvalues.append(PFC_Prod)
            
        t+=t_serie
        #  Partie Producion de la fonte dans le temps
        if t <= Temps_chargement:
            # Avant ou pendant Temps_chargement
            PFC_Prodvalues.append(PFC_Prod)
        else:
            # Après Temps_chargement
            if (t - Temps_chargement) % Temps_fusion == 0:
                numfours += 1
                # print(t,numfours)
                PFC_Prod += Fonte_produit
            PFC_Prodvalues.append(PFC_Prod)

        times.append(t)
        PFC_Consomvalues.append(PFC_Consom)
    tmax = t
    return times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax

In [49]:
nb_fours = 6 # 8-9 fours pour HWS, 6 fours pour DiSA
Temps_fusion = 45 # en min
Temps_chargement = 30 # 30 - 45  min

# Les valeurs d'entrée Input 

nb_moules_heure = 160 
t_serie = 5 # 5 min

PFCmin = 2500 # en kg
PFCmax = 4000 # en kg

times = [0] # en min
PFC_values = [4000] # en kg
ListeMasse_grappe = [35, 10, 20, 35] # en kg
ListeQuantite_mottes = [50, 50, 50, 50]

times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax = ProdConsom(ListeMasse_grappe, ListeQuantite_mottes, nb_moules_heure, t_serie, Temps_fusion, Temps_chargement)

In [50]:
Plot_consom(times, PFC_Consomvalues, tmin, tmax, PFCmin, PFCmax)

In [51]:
Plot_Prod(times, PFC_Prodvalues)

# Code 3

On a 4000 kg de fontes dans le four de Coulée avec un pourcentage de Mg = 0.0045 %.
On consonmme 36 Kg de fonte par minute dans le four de Coulée.
On perd 0.005 % de Mg par minute.

Tu es un mathématicien et tu dois Tracer la courbe d'évolution du Mg
dans le four de Coulée. 

Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!

In [1]:
import plotly.graph_objects as go
import numpy as np

def Plot_consom(times, PFC_values, tmin, tmax, PFCmin, PFCmax):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Add horizontal lines for PFCmin and PFCmax
    fig.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    
    fig.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")


    # Customize layout
    fig.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False,annotations=[
            dict(x=tmax,y=PFCmin,xref="x",yref="y",text="PFCmin",showarrow=True,arrowhead=2,ax=20,ay=-30),
            dict(x=tmax,y=PFCmax,xref="x",yref="y",text="PFCmax",showarrow=True,arrowhead=2,ax=20,ay=-30)])

    # Show the plot
    fig.show()

def Plot_Prod(times, PFC_values):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Customize layout
    fig.update_layout(title='Evolution de la fonte disponible dans le Four de Fusion au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False)

    # Show the plot
    fig.show()

In [432]:
def update_production_chargement_et_fusion(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod):
    
    """
    Met à jour la production de fonte en fonction du temps écoulé lors du chargement et de la fusion.
    
    :param t: Temps actuel en minutes
    :param Temps_chargement: Temps de chargement en minutes
    :param Temps_fusion: Temps de fusion en minutes
    :param numfours: Nombre de fours en production
    :param Fonte_produit: Quantité de fonte produite par cycle en kg
    :param PFC_Prod: Production actuelle de fonte en kg
    :param poche: Paramètre optionnel, par défaut à 0
    :return: Tuple (numfours, PFC_Prod) mis à jour
    """
    # if t >= Temps_chargement + Temps_fusion:
    if t >= temps_chargement + temps_fusion:
        numfours += 1
        PFC_Prod += fonte_produit
    return numfours, PFC_Prod

def calcule_chargement_et_fusion(times, PFC_Prodvalues, PFC_Consomvalues, fonte_produit, numfours, temps_chargement, temps_fusion):
    """
    Calcule la consommation et la production de fonte du début du chargement à la fin de la 1ere fusion.
    """

    PFC_Consom = PFC_Consomvalues[-1]
    PFC_Prod = PFC_Prodvalues[-1]
    t = times[-1]

    # Mise à jour de la production de fonte après chargement
    t += temps_chargement
    numfours, PFC_Prod = update_production_chargement_et_fusion(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
    PFC_Prodvalues.append(PFC_Prod)
    PFC_Consomvalues.append(PFC_Consom)
    times.append(t)


    # # Mise à jour de la production de fonte entre debut/fin fusion
    # t += temps_fusion-1
    # numfours, PFC_Prod = update_production_chargement_et_fusion(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
    # PFC_Prodvalues.append(PFC_Prod)
    # PFC_Consomvalues.append(PFC_Consom)
    # times.append(t)

    # # Mise à jour de la production de fonte après fusion
    # t += 1 
    # numfours, PFC_Prod = update_production_chargement_et_fusion(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
    # PFC_Prodvalues.append(PFC_Prod)
    # PFC_Consomvalues.append(PFC_Consom)
    # times.append(t)

    # Mise à jour de la production de fonte après fusion
    t += temps_fusion
    numfours, PFC_Prod = update_production_chargement_et_fusion(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
    PFC_Prodvalues.append(PFC_Prod)
    PFC_Consomvalues.append(PFC_Consom)
    times.append(t)

    return times, PFC_Prodvalues, PFC_Consomvalues


def update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche = 0):
    """
    Met à jour la production de fonte en fonction du temps écoulé.
    """
    if t > temps_chargement and (t - temps_chargement) % temps_fusion == 0 and numfours <= 4 :
        numfours += 1
        PFC_Prod += fonte_produit
    PFC_Prod -= poche
    return numfours, PFC_Prod


def calcule_Transitoire(times, PFC_Prodvalues, PFC_Consomvalues, fonte_produit, numfours, temps_chargement, temps_fusion):
    """
    Calcule la consommation et la production de fonte lors des 3 premieres poches (Régime Transitoire).
    """
    temps_poche_1, temps_poche_2, temps_poche_3 = 10, 10, 10 # temps traitement
    poche_1, poche_2, poche_3 = 1250, 1250, 1250

    PFC_Consom = PFC_Consomvalues[-1]
    PFC_Prod = PFC_Prodvalues[-1]
    t = times[-1]

    
    t += temps_poche_1
    PFC_Consom += poche_1 

    # Mise à jour de la production et consommation de fonte
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche_1)
    PFC_Prodvalues.append(PFC_Prod)
    PFC_Consomvalues.append(PFC_Consom)
    times.append(t)


    t += temps_poche_2
    # Mise à jour de la consommation de fonte
    PFC_Consom += poche_2 
    PFC_Consomvalues.append(PFC_Consom)
    
    # Mise à jour de la production de fonte
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche_2)
    PFC_Prodvalues.append(PFC_Prod)
    times.append(t)


    t += temps_poche_3
    # Mise à jour de la consommation de fonte
    PFC_Consom += poche_3 
    PFC_Consomvalues.append(PFC_Consom)
    
    # Mise à jour de la production de fonte
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche_3)
    PFC_Prodvalues.append(PFC_Prod)
    times.append(t)

    return times, PFC_Prodvalues,PFC_Consomvalues


def calculate_cadence_per_minute(nb_moules_heure, masse_grappe):
    """
    Calcule la cadence de consommation de fonte par minute.
    """
    return nb_moules_heure / 60 * masse_grappe


def calcule_temps_poche_4(PFC_Consomvalues, liste_masse_grappe, liste_quantite_mottes, Mgmax, Mgmin, PFCmin, eC):

    PFC_Consom = PFC_Consomvalues[-1]
    Mg = Mgmax
    
    temps_ajout_poche_i_Coulee = 0
    stop_boucle = False
    # Pour trouver le temps de la poche 4
    for masse_grappe, quantite_mottes in zip(liste_masse_grappe, liste_quantite_mottes):
        for _ in range(quantite_mottes):
            cadence =  calculate_cadence_per_minute(nb_moules_heure, masse_grappe)
            PFC_Consom -= cadence  
            Mg -= eC
            CondAjout = PFC_Consom  > PFCmin and Mg  > Mgmin
            if CondAjout : 
                temps_ajout_poche_i_Coulee += 1
            else:
                stop_boucle = True
                break
        if stop_boucle:
            break

    return temps_ajout_poche_i_Coulee


def calcule_temps_poche_i(PFC_Consom, index_grappe, liste_masse_grappe, index_motte, liste_quantite_mottes, Mg, Mgmin, PFCmin, eC):

    # print(PFC_Consom)


    liste_cadences = [7.83, 7.48, 2.96, 4.19, 7.83, 7.48, 2.96, 7.83, 5.63] # en Tonnes/hr
    liste_cadences = np.array(liste_cadences) * 1000 / 60
    liste_cadences = liste_cadences.tolist()
    liste_temps_serie = [3, 3, 3, 3, 3, 3, 3, 3] # en minutes
    liste_temps_serie = [0, 0, 0, 0, 0, 0, 0, 0] # en minutes



    liste_cadences = liste_cadences[index_grappe-1:]

    # print("Ici1",liste_quantite_mottes,liste_masse_grappe)

    liste_quantite_mottes = liste_quantite_mottes[index_grappe-1:]
    liste_masse_grappe = liste_masse_grappe[index_grappe-1:]

    # Cas ou fin d'une serie et Mais reste de serie encore
    if liste_quantite_mottes[0] == index_motte and len(liste_masse_grappe[index_grappe:]) != 0:
        liste_quantite_mottes = liste_quantite_mottes[index_grappe:]
        liste_masse_grappe = liste_masse_grappe[index_grappe:]
        # print("Ici1",liste_quantite_mottes,liste_masse_grappe)
    
    else :
        liste_quantite_mottes[0] -=  index_motte
        # print("Ici2",liste_quantite_mottes,liste_masse_grappe)


    # print("Ici2",liste_quantite_mottes,liste_masse_grappe)


    temps_ajout_poche_i_Coulee = 0
    stop_boucle = False
    # Pour trouver le temps de la poche 4
    for idx, (masse_grappe, quantite_mottes) in enumerate(zip(liste_masse_grappe, liste_quantite_mottes)):
        cadence = liste_cadences[idx]
        temps_serie = liste_temps_serie[idx]

        for _ in range(quantite_mottes):
            PFC_Consom -= cadence  
            Mg -= eC
            # print(Mg,PFC_Consom)
            if PFC_Consom  > PFCmin and Mg  > Mgmin : 
                print("index_grappe, PFC_Consom",index_grappe,PFC_Consom)
                temps_ajout_poche_i_Coulee += 1
                # print("temps_ajout_poche_i_Coulee ",temps_ajout_poche_i_Coulee)
            else :
                # print(Mg,PFC_Consom)
                stop_boucle = True
                break
        if stop_boucle:
            break
        else :
            # print("temps_ajout_poche_i_Coulee ",temps_ajout_poche_i_Coulee)
            temps_ajout_poche_i_Coulee += temps_serie
            
    # print("temps_ajout_poche_i_Coulee ",temps_ajout_poche_i_Coulee)

    return temps_ajout_poche_i_Coulee



In [429]:


def ProdConsom(liste_masse_grappe, liste_quantite_mottes, nb_moules_heure, t_serie, temps_fusion, temps_chargement, temps_traitement, Mg, Mgmin,Mgmax, eC):
    """
    Calcule la consommation et la production de fonte.
    """
    # Initialisation des variables
    numfours = 1
    fonte_produit = 5000  # Fonte produite toutes les 45 min en kg
    PFC_Prodvalues = [0]  # Production de fonte en kg
    PFC_Consomvalues = [0]  # Consommation de fonte en kg
    times = [0]  # Temps en minutes

    Listetemps_ajout_poche_i_Coulee = []
    Listetemps_depart_poche_i_Fusion = []

    times, PFC_Prodvalues,PFC_Consomvalues = calcule_chargement_et_fusion(times, PFC_Prodvalues, PFC_Consomvalues, fonte_produit, numfours, temps_chargement, temps_fusion)

    times, PFC_Prodvalues,PFC_Consomvalues = calcule_Transitoire(times, PFC_Prodvalues, PFC_Consomvalues, fonte_produit, numfours, temps_chargement, temps_fusion)

    temps_ajout_poche_4_Coulee = calcule_temps_poche_4(PFC_Consomvalues, liste_masse_grappe, liste_quantite_mottes, Mgmax, Mgmin, PFCmin, eC)
    temps_depart_poche_i_Fusion = temps_ajout_poche_4_Coulee - temps_traitement
    Listetemps_ajout_poche_i_Coulee.append(temps_ajout_poche_4_Coulee)
    Listetemps_depart_poche_i_Fusion.append(temps_depart_poche_i_Fusion)
    
    numfours += 1




    PFC_Prod = PFC_Prodvalues[-1]
    PFC_Consom = PFC_Consomvalues[-1]
    t = times[-1]

    t_avantpoche_i = t
    poche_i = 1250

    # print( "ICI",PFC_Consom,t,Listetemps_ajout_poche_i_Coulee)


# ----------------------------------------------------------------------------------

    # Boucle principale consomation et production
    # Indices de suivi pour les listes
    t_max = 600 #4*60
    index_grappe = 1 #0
    index_motte = 1 #0
    quantite_mottes = liste_quantite_mottes[index_grappe-1]

    liste_cadences = [7.83, 7.48, 2.96, 4.19, 7.83, 7.48, 2.96, 7.83, 5.63] # en Tonnes/hr
    liste_cadences = np.array(liste_cadences) * 1000 / 60
    liste_cadences = liste_cadences.tolist()

    while t < t_max:  
        if index_grappe <= len(liste_masse_grappe) and index_motte <= quantite_mottes:
            masse_grappe = liste_masse_grappe[index_grappe-1]
            cadence =  liste_cadences[index_grappe-1]
            
            # print("Changement de serie ",index_grappe,index_motte)
            # print(PFC_Consom,t,Listetemps_ajout_poche_i_Coulee)

            # Avant de consommer de la fonte, on doit savoir :
            # Quand prendre la poche_i dans le Fours de Fusion ?
            if t == (Listetemps_depart_poche_i_Fusion[-1] + t_avantpoche_i):
                numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche_i)
            else:
                numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)

            # print( "ICI1",PFC_Consom,cadence,t,t_avantpoche_i,Listetemps_ajout_poche_i_Coulee)

            # Avant de consommer de la fonte, on doit savoir :
            # Quand ajouter la poche_i dans le Fours de Coulée ?
            if t == (Listetemps_ajout_poche_i_Coulee[-1] + t_avantpoche_i):
                PFC_Consom += poche_i
                Mg = Mgmax

                # print( "ICI2",PFC_Consom,cadence,t,t_avantpoche_i,Listetemps_ajout_poche_i_Coulee)

                # Calcul Temps prochain poche_i
                temps_ajout_poche_i_Coulee = calcule_temps_poche_i(PFC_Consom, index_grappe, liste_masse_grappe, index_motte, liste_quantite_mottes, Mg, Mgmin, PFCmin, eC)
                Listetemps_ajout_poche_i_Coulee.append(temps_ajout_poche_i_Coulee)
                t_avantpoche_i = t

                temps_depart_poche_i_Fusion = Listetemps_ajout_poche_i_Coulee[-1] - temps_traitement
                Listetemps_depart_poche_i_Fusion.append(temps_depart_poche_i_Fusion)


            # Mise à jour de la consommation de fonte et de Mg
            PFC_Consom -= cadence  
            Mg -= eC

            # Mise à jour des indices de suivi des mottes
            if index_motte  < quantite_mottes:
                index_motte += 1
            else:
                # Gestion du Changement de Serie
                # print("Changement de serie 2")

                # print("la", index_grappe)

                index_motte = 1
                index_grappe += 1
                if index_grappe <= len(liste_quantite_mottes):
                    quantite_mottes = liste_quantite_mottes[index_grappe-1]





        # Mise à jour du temps
        t += 1
        # print("t = ",t)
        times.append(t)
        PFC_Prodvalues.append(PFC_Prod)
        PFC_Consomvalues.append(PFC_Consom)



    tmin = times[0]
    tmax = times[-1]
    return times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax


In [433]:
nb_fours = 6 # 8-9 fours pour HWS, 6 fours pour DiSA
temps_fusion = 45 # en min
temps_chargement = 30 # 30 - 45  min

# Les valeurs d'entrée Input 

nb_moules_heure = 190 
t_serie = 5 # 5 min

PFCmin = 2500 # en kg
PFCmax = 4000 # en kg

Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
temps_traitement = 10  # en  min



Mg = Mgmax

liste_masse_grappe = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65] # en kg
liste_quantite_mottes = [121, 50, 35, 10, 100, 50, 35, 50, 40]

liste_cadences = [7.83, 7.48, 2.96, 4.19, 7.83, 7.48, 2.96, 7.83, 5.63] # en Tonnes/hr

liste_temps_changement_serie = [3, 3, 3, 3, 3, 3, 3, 3] # en minutes


# liste_masse_grappe = [41.2, 39.36, 15.6] # en kg
# liste_quantite_mottes = [1*121, 50, 35]


# liste_masse_grappe = [35, 35] # en kg
# liste_quantite_mottes = [50, 50]

# liste_masse_grappe = [35] # en kg
# liste_quantite_mottes = [200]

times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax = ProdConsom(liste_masse_grappe, liste_quantite_mottes, nb_moules_heure, t_serie, temps_fusion, temps_chargement, temps_traitement, Mg, Mgmin,Mgmax, eC)

index_grappe, PFC_Consom 1 3695.0
index_grappe, PFC_Consom 1 3564.5
index_grappe, PFC_Consom 1 3434.0
index_grappe, PFC_Consom 1 3303.5
index_grappe, PFC_Consom 1 3173.0
index_grappe, PFC_Consom 1 3042.5
index_grappe, PFC_Consom 1 2912.0
index_grappe, PFC_Consom 1 2781.5
index_grappe, PFC_Consom 1 2651.0
index_grappe, PFC_Consom 1 2520.5
index_grappe, PFC_Consom 1 3640.0
index_grappe, PFC_Consom 1 3509.5
index_grappe, PFC_Consom 1 3379.0
index_grappe, PFC_Consom 1 3248.5
index_grappe, PFC_Consom 1 3118.0
index_grappe, PFC_Consom 1 2987.5
index_grappe, PFC_Consom 1 2857.0
index_grappe, PFC_Consom 1 2726.5
index_grappe, PFC_Consom 1 2596.0
index_grappe, PFC_Consom 1 3715.5
index_grappe, PFC_Consom 1 3585.0
index_grappe, PFC_Consom 1 3454.5
index_grappe, PFC_Consom 1 3324.0
index_grappe, PFC_Consom 1 3193.5
index_grappe, PFC_Consom 1 3063.0
index_grappe, PFC_Consom 1 2932.5
index_grappe, PFC_Consom 1 2802.0
index_grappe, PFC_Consom 1 2671.5
index_grappe, PFC_Consom 1 2541.0
index_grappe, 

In [434]:
Plot_Prod(times, PFC_Prodvalues)
Plot_consom(times, PFC_Consomvalues, tmin, tmax, PFCmin, PFCmax)